# Train NanoDet with custom dataset
<a target="_blank" href="https://colab.research.google.com/github/SonySemiconductorSolutions/aitrios-rpi-tutorials-ai-model-training/blob/main/notebooks/nanodet-ppe/custom_nanodet.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Training NanoDet model to detect Personal Protection Equipment (PPE) using open source dataset.

Nanodet training based on https://github.com/RangiLyu/nanodet/tree/main

Tutorial includes:
- Dataset setup
- Nanodet model setup
- Training
- Quantization using [Model Compression Toolkit - MCT](https://github.com/sony/model_optimization)

In [1]:
!pip install -q --no-cache-dir torch~=1.13.1 torchvision "tensorflow>=2.14.0,<2.15.0" pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 237.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 157.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 198.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 284.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 251.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 189.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 194.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 292.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 202.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 262.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 338.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 244.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver d

In [2]:
# Perform initial checks in order to continue
import shutil
import tensorflow as tf
import torch

assert '2.14' in tf.__version__, print(tf.__version__)
assert '1.13' in torch.__version__, print(torch.__version__)

# Check shared memory
shm_stats = shutil.disk_usage('/dev/shm')
shm_in_gb = shm_stats.total / (1024 ** 3)
print(f"shm memory: {shm_in_gb:.2f}GB")

print(f'Is cuda available: {torch.cuda.is_available()}')
assert shm_in_gb >= 12 or torch.cuda.is_available()

shm memory: 5.68GB
Is cuda available: True


# Installation

In [3]:
"""
Known errors:
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.13.1 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 1.13.1 which is incompatible.
"""
NANODET_COMMIT = 'be9b4a9'
!rm -rf nanodet
!git clone https://github.com/RangiLyu/nanodet.git
!touch nanodet/nanodet/model/__init__.py
!cd nanodet && git checkout {NANODET_COMMIT} && pip install -q --no-cache-dir -r requirements.txt

Cloning into 'nanodet'...
remote: Enumerating objects: 2722, done.
remote: Counting objects: 100% (616/616), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 2722 (delta 535), reused 485 (delta 485), pack-reused 2106 (from 1)
Receiving objects: 100% (2722/2722), 5.28 MiB | 5.60 MiB/s, done.
Resolving deltas: 100% (1618/1618), done.
Note: switching to 'be9b4a9'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at be9b4a9 Replaced opencv with imagesize to get the actual image size (#548)
     ━━

# Dataset
- go to https://universe.roboflow.com/ai-camp-safety-equipment-detection/ppe-detection-using-cv/dataset/3 and click `"Download Dataset"`
- choose format `"COCO"` and `"show download code"` and `"continue"`
- choose `"Terminal"` and copy the command `"curl..."` and paste the command in the cell below.
- add `"!"` in the beginning of the command and replace `"\&gt;"` with `">"`

In [6]:
# Add below your download code from Roboflow, it should look like the following, with your unique roboflow dataset url:
# Example (with "!" added in the beginning of the command and replaced "&gt;" with ">". Also added "-q" for less output):
!curl -L "https://universe.roboflow.com/ds/4juCFTKWpx?key=HlRho6YNW4" > roboflow.zip; unzip -q roboflow.zip; rm roboflow.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   901  100   901    0     0   2160      0 --:--:-- --:--:-- --:--:--  2165
100  235M  100  235M    0     0  13.8M      0  0:00:17  0:00:17 --:--:-- 17.4M


In [7]:
# Move test/train/valid to dataset folder
from pathlib import Path
DATASET_PATH = 'dataset/PPE_Detection_Using_CV.v3i.coco'
if not Path(f'{DATASET_PATH}/train/_annotations.coco.json').exists():
    assert Path(f'train/_annotations.coco.json').exists()
    assert Path(f'valid/_annotations.coco.json').exists()
    assert Path(f'test/_annotations.coco.json').exists()
    !mkdir -p $DATASET_PATH
    !mv test train valid *txt $DATASET_PATH/

In [8]:
assert Path(f'{DATASET_PATH}/train/_annotations.coco.json').exists()
assert Path(f'{DATASET_PATH}/valid/_annotations.coco.json').exists()

# Training config file: nanodet-plus-m-1.5x_416-ppe.yml
The following block of code creates the NanoDet training config file which
is based on nanodet/config/nanodet-plus-m-1.5x_416.yml.
Updated for the custom PPE dataset
Change number of `total_epochs` for better performance.

If training on GPU, then set   `gpu_ids`:
 * 1 gpu: [0]
 * 2 gpu: [0,1]
 * etc...

Increase `total_epochs`, for example 20.

Feel free to increase `val_intervals`, for example 10.

For details see NanoDet github repo and [config docs](https://github.com/RangiLyu/nanodet/blob/main/docs/config_file_detail.md). Observe recommendation to adjust `lr` with `batch_size`.

In [28]:
%%bash
touch nanodet-plus-m-1.5x_416-ppe.yml
cat <<EOF >nanodet-plus-m-1.5x_416-ppe.yml
# Comments:
# -  based on nanodet/config/nanodet-plus-m-1.5x_416.yml
# -  "device": settings for colab T4 GPU
# -  "total_epochs": set to 20 during testing, default 300
save_dir: workspace/nanodet-plus-m-1.5x_416-ppe
model:
  weight_averager:
    name: ExpMovingAverager
    decay: 0.9998
  arch:
    name: NanoDetPlus
    detach_epoch: 10
    backbone:
      name: ShuffleNetV2
      model_size: 1.5x
      out_stages: [2,3,4]
      activation: LeakyReLU
    fpn:
      name: GhostPAN
      in_channels: [176, 352, 704]
      out_channels: 128
      kernel_size: 5
      num_extra_level: 1
      use_depthwise: True
      activation: LeakyReLU
    head:
      name: NanoDetPlusHead
      num_classes: 8
      input_channel: 128
      feat_channels: 128
      stacked_convs: 2
      kernel_size: 5
      strides: [8, 16, 32, 64]
      activation: LeakyReLU
      reg_max: 7
      norm_cfg:
        type: BN
      loss:
        loss_qfl:
          name: QualityFocalLoss
          use_sigmoid: True
          beta: 2.0
          loss_weight: 1.0
        loss_dfl:
          name: DistributionFocalLoss
          loss_weight: 0.25
        loss_bbox:
          name: GIoULoss
          loss_weight: 2.0
    # Auxiliary head, only use in training time.
    aux_head:
      name: SimpleConvHead
      num_classes: 8
      input_channel: 256
      feat_channels: 256
      stacked_convs: 4
      strides: [8, 16, 32, 64]
      activation: LeakyReLU
      reg_max: 7
data:
  train:
    name: CocoDataset
    img_path: dataset/PPE_Detection_Using_CV.v3i.coco/train
    ann_path: dataset/PPE_Detection_Using_CV.v3i.coco/train/_annotations.coco.json
    input_size: [416,416] #[w,h]
    keep_ratio: False
    pipeline:
      perspective: 0.0
      scale: [0.6, 1.4]
      stretch: [[0.8, 1.2], [0.8, 1.2]]
      rotation: 0
      shear: 0
      translate: 0.2
      flip: 0.5
      brightness: 0.2
      contrast: [0.6, 1.4]
      saturation: [0.5, 1.2]
      normalize: [[103.53, 116.28, 123.675], [57.375, 57.12, 58.395]]
  val:
    name: CocoDataset
    img_path: dataset/PPE_Detection_Using_CV.v3i.coco/valid
    ann_path: dataset/PPE_Detection_Using_CV.v3i.coco/valid/_annotations.coco.json
    input_size: [416,416] #[w,h]
    keep_ratio: False
    pipeline:
      normalize: [[103.53, 116.28, 123.675], [57.375, 57.12, 58.395]]
device:
  gpu_ids: -1
  workers_per_gpu: 2
  batchsize_per_gpu: 32
  precision: 32 # set to 16 to use AMP training
schedule:
#  resume:
#  load_model:
  optimizer:
    name: AdamW
    lr: 0.001
    weight_decay: 0.05
  warmup:
    name: linear
    steps: 500
    ratio: 0.0001
  total_epochs: 20
  lr_schedule:
    name: CosineAnnealingLR
    T_max: 300
    eta_min: 0.00005
  val_intervals: 10
grad_clip: 35
evaluator:
  name: CocoDetectionEvaluator
  save_key: mAP
log:
  interval: 10

class_names: [
  'safety-equipment',
  'person',
  'goggles',
  'helmet',
  'no-goggles',
  'no-helmet',
  'no-vest',
  'vest']
EOF

# Training

In [31]:
# OBSERVE: update the following assert statement to match your yml file settings.

import yaml
with open('nanodet-plus-m-1.5x_416-ppe.yml', 'r') as file:
    config = yaml.safe_load(file)

assert config['device']['gpu_ids'] == -1, print(f"gpu_ids: {config['device']['gpu_ids']}")
assert config['schedule']['total_epochs'] == 20, print(f"total_epochs: {config['schedule']['total_epochs']}")
assert config['schedule']['val_intervals'] == 10, print(f"val_intervals: {config['schedule']['val_intervals']}")

In [32]:
import torch
assert '1.13' in torch.__version__, print(torch.__version__)
assert Path('nanodet-plus-m-1.5x_416-ppe.yml').exists()
!export PYTHONPATH=$PWD/nanodet:$PYTHONPATH && python nanodet/tools/train.py nanodet-plus-m-1.5x_416-ppe.yml

[NanoDet][12-21 14:23:06]INFO:Setting up data...
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
[NanoDet][12-21 14:23:06]INFO:Creating model...
model size is  1.5x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1_5-3c479a10.pth
Finish initialize NanoDet-Plus Head.
[NanoDet][12-21 14:23:06]INFO:Using CPU training
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name      | Type        | Params
------------------------------------------
0 | model     | NanoDetPlus | 7.8 M 
1 | avg_model | NanoDet

# Remove aux layers that are only used during training

In [23]:
import sys
sys.path.insert(0,"./nanodet")

import copy
import torch
from nanodet.model.arch import build_model
from nanodet.util import cfg, load_config, Logger

def remove_aux(cfg, model_path, remove_layers=['aux_fpn', 'aux_head'], debug=False):
    model = build_model(cfg.model)
    ckpt = torch.load(model_path, map_location=lambda storage, loc: storage)
    if len(remove_layers) > 0:
        state_dict = copy.deepcopy(ckpt['state_dict'])
        for rlayer in remove_layers:
            for layer in ckpt['state_dict']:
                if rlayer in layer:
                    del state_dict[layer]
                    if debug:
                        print(f'removed layer: {layer}')
        del ckpt['state_dict']
        ckpt['state_dict'] = copy.deepcopy(state_dict)
        del state_dict
    return ckpt

In [24]:
config_path = 'nanodet-plus-m-1.5x_416-ppe.yml'
model_path = 'workspace/nanodet-plus-m-1.5x_416-ppe/model_best/nanodet_model_best.pth'
dst_path = 'workspace/nanodet-plus-m-1.5x_416-ppe/model_best/nanodet_model_best-removed-aux.pth'

load_config(cfg, config_path)
ckpt = remove_aux(cfg, model_path, ['aux_fpn', 'aux_head'])
torch.save(ckpt, dst_path)
print(f'Saved to: {dst_path}')

model size is  1.5x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1_5-3c479a10.pth
Finish initialize NanoDet-Plus Head.


FileNotFoundError: [Errno 2] No such file or directory: 'workspace/nanodet-plus-m-1.5x_416-ppe/model_best/nanodet_model_best.pth'

In [14]:
# Compare size w and w/o aux
!ls -l workspace/nanodet-plus-m-1.5x_416-ppe/model_best

ls: cannot access 'workspace/nanodet-plus-m-1.5x_416-ppe/model_best': No such file or directory


# Quantization of custom NanoDet model using Model Compression Toolkit
Quantization is based on https://github.com/sony/model_optimization/blob/v2.0.0/tutorials/notebooks/keras/ptq/example_keras_nanodet_plus.ipynb

# Installation

In [ ]:
!pip install --no-cache-dir -q "tensorflow>=2.14.0,<2.15.0" pycocotools
import sys
MCT_COMMIT = 'v2.1.0'
!rm -rf local_mct
!git clone https://github.com/sony/model_optimization.git local_mct && cd local_mct && git checkout {MCT_COMMIT} && pip install --no-cache-dir -r requirements.txt
sys.path.insert(0,"./local_mct")

# Keras NanoDet float model

In [ ]:
from pathlib import Path

import numpy as np
import tensorflow as tf
import torch
assert '2.14' in tf.__version__, print(tf.__version__)
assert '1.13' in torch.__version__, print(torch.__version__)

from keras.models import Model
import model_compression_toolkit as mct
from tutorials.mct_model_garden.models_keras.nanodet.nanodet_keras_model import nanodet_plus_m
from tutorials.mct_model_garden.models_keras.utils.torch2keras_weights_translation import load_state_dict
from tutorials.mct_model_garden.models_keras.nanodet.nanodet_keras_model import nanodet_box_decoding
assert 'local_mct' in mct.__file__, print(mct.__file__)


In [ ]:
# Upload the trained custom model
CUSTOM_WEIGHTS_FILE = dst_path  # The NanoDet model trained with PPE dataset
CLASS_NAMES = [
  'safety-equipment',
  'person',
  'goggles',
  'helmet',
  'no-goggles',
  'no-helmet',
  'no-vest',
  'vest']
NUM_CLASSES = len(CLASS_NAMES)

DATASET_TRAIN = 'dataset/PPE_Detection_Using_CV.v3i.coco/train'
ANNOT_TRAIN = 'dataset/PPE_Detection_Using_CV.v3i.coco/train/_annotations.coco.json'
DATASET_VALID = 'dataset/PPE_Detection_Using_CV.v3i.coco/valid'
ANNOT_VALID = 'dataset/PPE_Detection_Using_CV.v3i.coco/valid/_annotations.coco.json'
DATASET_REPR = DATASET_VALID
ANNOT_REPR = ANNOT_VALID

QUANTIZED_MODEL_NAME = 'nanodet-quant-ppe.keras'

BATCH_SIZE = 5
N_ITER = 20  # 1 for testing, otherwise 20

assert Path(CUSTOM_WEIGHTS_FILE).exists()
assert Path(DATASET_REPR).exists()

In [ ]:

def get_model(weights=CUSTOM_WEIGHTS_FILE, num_classes=NUM_CLASSES):
    INPUT_RESOLUTION = 416
    INPUT_SHAPE = (INPUT_RESOLUTION, INPUT_RESOLUTION, 3)
    SCALE_FACTOR = 1.5
    BOTTLENECK_RATIO = 0.5
    FEATURE_CHANNELS = 128

    pretrained_weights = torch.load(weights, map_location=torch.device('cpu'))['state_dict']
    # Generate Nanodet base model
    model = nanodet_plus_m(INPUT_SHAPE, SCALE_FACTOR, BOTTLENECK_RATIO, FEATURE_CHANNELS, num_classes)

    # Set the pre-trained weights
    load_state_dict(model, state_dict_torch=pretrained_weights)

    # Add Nanodet Box decoding layer (decode the model outputs to bounding box coordinates)
    scores, boxes = nanodet_box_decoding(model.output, res=INPUT_RESOLUTION, num_classes=num_classes)

    # Add TensorFlow NMS layer
    outputs = tf.image.combined_non_max_suppression(
        boxes,
        scores,
        max_output_size_per_class=300,
        max_total_size=300,
        iou_threshold=0.65,
        score_threshold=0.001,
        pad_per_class=False,
        clip_boxes=False
        )

    model = Model(model.input, outputs, name='Nanodet_plus_m_1.5x_416')

    print('Model is ready for evaluation')
    return model

In [ ]:
# known warning:  WARNING: head.distribution_project.project not assigned to keras model !!!
float_model = get_model(CUSTOM_WEIGHTS_FILE, NUM_CLASSES)

# PTQ quantization

In [ ]:
from typing import Callable, Iterator, Tuple, List

import cv2
from tutorials.mct_model_garden.evaluation_metrics.coco_evaluation import coco_dataset_generator, CocoEval

def nanodet_preprocess(x):
    img_mean = [103.53, 116.28, 123.675]
    img_std = [57.375, 57.12, 58.395]
    x = cv2.resize(x, (416, 416))
    x = (x - img_mean) / img_std
    return x

def get_representative_dataset(n_iter: int, dataset_loader: Iterator[Tuple]):
    def representative_dataset() -> Iterator[List]:
        ds_iter = iter(dataset_loader)
        for _ in range(n_iter):
            yield [next(ds_iter)[0]]

    return representative_dataset

def quantization(float_model, dataset, annot, n_iter=N_ITER):
    # Load representative dataset
    representative_dataset = coco_dataset_generator(dataset_folder=dataset,
                                                    annotation_file=annot,
                                                    preprocess=nanodet_preprocess,
                                                    batch_size=BATCH_SIZE)

    tpc = mct.get_target_platform_capabilities('tensorflow', 'imx500')

    # Preform post training quantization
    quant_model, _ = mct.ptq.keras_post_training_quantization(
        float_model,
        representative_data_gen=get_representative_dataset(n_iter, representative_dataset),
        target_platform_capabilities=tpc)

    print('Quantized model is ready')
    return quant_model

In [ ]:
quant_model = quantization(float_model, DATASET_REPR, ANNOT_REPR)
print(f'Representative dataset: {DATASET_REPR}')

In [ ]:
# Observe that loading quantized model might require specification of custom layers,
# see https://github.com/sony/model_optimization/issues/1104
quant_model.save(QUANTIZED_MODEL_NAME)
print(f'Quantized model saved: {QUANTIZED_MODEL_NAME}')

_todo_: coco evaluation of the custom quantized NanoDet model requires some update to the mct repo.

# Visualize detection

In [ ]:
# Helper functions for visualization
import numpy as np

def load_and_preprocess_image(image_path: str, preprocess: Callable) -> np.ndarray:
    """
    Load and preprocess an image from a given file path.

    Args:
        image_path (str): Path to the image file.
        preprocess (function): Preprocessing function to apply to the loaded image.

    Returns:
        numpy.ndarray: Preprocessed image.
    """
    image = cv2.imread(image_path)
    image = preprocess(image)
    image = np.expand_dims(image, 0)
    return image

# draw a single bounding box onto a numpy array image
def draw_bounding_box(img, annotation, scale, class_id, score):
    row = scale[0]
    col = scale[1]
    x_min, y_min = int(annotation[1]*col), int(annotation[0]*row)
    x_max, y_max = int(annotation[3]*col), int(annotation[2]*row)

    color = (0,255,0)

    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, 2)
    text = f'{int(class_id)}: {score:.2f}'
    cv2.putText(img, text, (x_min + 10, y_min + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# draw all annotation bounding boxes on an image
def annotate_image(img, output, scale, quantized_model=False, threshold=0.55):
    if quantized_model:
        b = output[0].numpy()[0]
        s = output[1].numpy()[0]
        c = output[2].numpy()[0]
    else:
        print('Assuming float model')
        b = output.nmsed_boxes.numpy()[0]
        s = output.nmsed_scores.numpy()[0]
        c = output.nmsed_classes.numpy()[0]
    for index, row in enumerate(b):
        if s[index] > threshold:
            #print(f'row: {row}')
            id = int(c[index])
            draw_bounding_box(img, row, scale, id, s[index])
            print(f'class: {CLASS_NAMES[id]} ({id}), score: {s[index]:.2f}')
    return {'bbox':b, 'score':s, 'classes':c}

In [ ]:
# See appendix for results. For 2 epochs, the bounding boxes are not perfect...
# But improves considerably for 20 epochs.
test_img = 'dataset/PPE_Detection_Using_CV.v3i.coco/valid/image_257_jpg.rf.1a3a6eb456134cce302712c109645c26.jpg'
img = load_and_preprocess_image(f'{test_img}', nanodet_preprocess)
output = quant_model(img)
image = cv2.imread(f'{test_img}')
print(f'image shape: {image.shape}')
r = annotate_image(image, output, scale=image.shape, quantized_model=True)
assert r['score'][0] > 0.5, print(f"r['score'][0] > 0.5 failed: {r['score'][0]}")
dst = f'annotated.jpg'
if cv2.imwrite(dst, image):
    print(f'Annotated image saved to: {dst}')
else:
    print(f'Failed saving annotated image')
from matplotlib import pyplot as plt
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

# Next step
__OBSERVE__: First, save the quantized model to your local machine. You will need it for the conversion and packaging steps.

Next step is to convert and package the model for IMX500. _todo: link to further instructions. This model requires bgr settings in the post-converter._

# Appendix
## Results total_epochs=2
```
[NanoDet][07-12 10:47:40]INFO:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.191
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.011
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.045
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.076
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.100
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.207
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.224
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.017
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.149
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.271

[NanoDet][07-12 10:47:40]INFO:
| class            | AP50   | mAP   | class     | AP50   | mAP   |
|:-----------------|:-------|:------|:----------|:-------|:------|
| Safety-Equipment | nan    | nan   | Person    | 54.1   | 20.1  |
| goggles          | 1.6    | 0.4   | helmet    | 40.3   | 13.4  |
| no-goggles       | 6.9    | 2.3   | no-helmet | 0.0    | 0.0   |
| no-vest          | 3.9    | 1.0   | vest      | 27.1   | 7.9   |
[NanoDet][07-12 10:47:40]INFO:Saving model to workspace/nanodet-plus-m-1.5x_416-ppe/model_best/nanodet_model_best.pth
[NanoDet][07-12 10:47:40]INFO:Val_metrics: {'mAP': 0.06442128900684024, 'AP_50': 0.1912998265318579, 'AP_75': 0.026441697602184976, 'AP_small': 0.010583883892274994, 'AP_m': 0.044976540581496194, 'AP_l': 0.0756733533889817}
`Trainer.fit` stopped: `max_epochs=2` reached.
```
## Results total_epochs=20
```
Comments:
-  based on nanodet/config/nanodet-plus-m-1.5x_416.yml
-  "device": settings for colab T4 GPU
-  "total_epochs": set to 20 during testing, default 300
...
[NanoDet][05-16 09:25:43]INFO:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.301
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.610
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.250
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.078
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.212
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.362
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.281
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.479
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.497
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.185
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.414
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.570

[NanoDet][05-16 09:25:43]INFO:
| class            | AP50   | mAP   | class     | AP50   | mAP   |
|:-----------------|:-------|:------|:----------|:-------|:------|
| Safety-Equipment | nan    | nan   | Person    | 86.4   | 49.3  |
| goggles          | 35.7   | 15.4  | helmet    | 86.0   | 46.3  |
| no-goggles       | 42.6   | 16.6  | no-helmet | 34.2   | 13.8  |
| no-vest          | 59.6   | 26.3  | vest      | 82.5   | 42.8  |
[NanoDet][05-16 09:25:44]INFO:Saving model to workspace/nanodet-plus-m-1.5x_416-ppe/model_best/nanodet_model_best.pth
[NanoDet][05-16 09:25:44]INFO:Val_metrics: {'mAP': 0.3006027561087712, 'AP_50': 0.6099170448933922, 'AP_75': 0.2496291506747232, 'AP_small': 0.07788513248169772, 'AP_m': 0.212229159695, 'AP_l': 0.36198435595574324}
`Trainer.fit` stopped: `max_epochs=20` reached.

real	21m35.722s
user	25m48.699s
sys	6m6.849s
```